# Seq2Seq LLM Reprompt




In [1]:
from datasets import load_dataset
import pandas as pd 

# df.columns
ds = load_dataset("csv",data_files={"train":'gemma10000.csv'})

/home/user/envs/test_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# df=df[['original_text','gemma_7b_rewritten_text_temp0','rewrite_prompt']]
# df['diff_prompt']='Tell me what prompt converted the following text: \n"""' + df['original_text'] + '"""\n to this text \n"""' + df['gemma_7b_rewritten_text_temp0'] + '"""\n'
#original text prefix

orig_prefix = "summarize: Original Text:"

rewrite_prefix = "\nRewritten Text:"

import json
import re

def get_prompt(orig_text, transformed_text):
    message = f"{orig_prefix} {orig_text} {rewrite_prefix} {transformed_text}"
    #message = f"{rewrite_prefix} {transformed_text}"
    return message


# interate over all the rows formate the dataset and store it in a jsonl file
def process_jsonl_file(output_file_path):
    with open(output_file_path, "w") as output_jsonl_file:
        for item in ds["train"]:
            json_object = {
                #"text": get_prompt(item["original_text"],item['gemma_7b_rewritten_text_temp0']),
                "text": get_prompt(item["original_text"],item['rewritten_text']),
                "output": item['rewrite_prompt']
            }
            output_jsonl_file.write(json.dumps(json_object) + "\n")

# Provide the path where you want to save the formatted dataset
process_jsonl_file("./training_dataset.jsonl")


In [3]:
from datasets import Dataset
train_dataset = load_dataset('json', data_files='./training_dataset.jsonl', split="train")

train_dataset=train_dataset.train_test_split(test_size=0.1, seed=37)


Generating train split: 9999 examples [00:00, 205930.84 examples/s]


In [4]:
model_checkpoint = "google/flan-t5-large"
#model_checkpoint = "facebook/bart-large"

In [5]:
from datasets import load_dataset
from evaluate import load

metric = load("rouge")

2024-03-25 14:19:14.024074: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-25 14:19:14.024114: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-25 14:19:14.025148: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-25 14:19:14.030256: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-25 14:19:14.664045: W tensorflow/compiler/tf2

In [6]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, device_map="auto", trust_remote_code=True)

In [7]:
#tokenizer.save_pretrained("./Seq2Seq_"+model_name)

('./Seq2Seq_bart-large/tokenizer_config.json',
 './Seq2Seq_bart-large/special_tokens_map.json',
 './Seq2Seq_bart-large/vocab.json',
 './Seq2Seq_bart-large/merges.txt',
 './Seq2Seq_bart-large/added_tokens.json',
 './Seq2Seq_bart-large/tokenizer.json')

In [8]:
# from transformers import AutoConfig

# config=AutoConfig.from_pretrained(model_checkpoint)
# config.save_pretrained('./Seq2Seq_'+model_name)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [9]:
# config

BartConfig {
  "_name_or_path": "facebook/bart-large",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 1024,
  "mode

In [8]:
max_input_length = 1024
max_target_length = 16

In [9]:
def preprocess_function(examples):
    model_inputs = tokenizer(examples["text"], max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["output"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
tokenized_dataset = train_dataset.map(preprocess_function, batched=True)

Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 6682.69 examples/s]


In [11]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'output', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 8999
    })
    test: Dataset({
        features: ['text', 'output', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})

In [13]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, device_map="auto")

In [14]:
batch_size = 2

args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-reprompt",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    #gradient_accumulation_steps=4,
    #gradient_checkpointing=True,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=20,
    save_strategy="epoch",
    predict_with_generate=True,
    #fp16=False,
    #tf32=True,
    #debug="underflow_overflow",
    load_best_model_at_end=True
)

In [15]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [16]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [17]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],    
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/home/user/envs/test_env/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


ValueError: You cannot perform fine-tuning on purely quantized models. Please attach trainable adapters on top of the quantized model to correctly perform fine-tuning. Please see: https://huggingface.co/docs/transformers/peft for more details

In [17]:
import torch
torch.cuda.empty_cache()
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.619900,0.492699,54.987700,39.325300,54.715400,54.679200,12.747000


/home/user/envs/test_env/lib/python3.10/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
trainer.save_model(save_loc)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(save_loc, device_map="auto")
#tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large', device_map="auto")
model = AutoModelForSeq2SeqLM.from_pretrained(save_loc, device_map="auto")

In [ ]:
def pred(x):
    input_ids = tokenizer(x, return_tensors='pt').input_ids.to('cuda')
    outputs = model.generate(input_ids, max_new_tokens=max_target_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
print(pred(train_dataset["train"][0]["text"]))
print(train_dataset["train"][0]["output"])
print(train_dataset["train"][0]["text"])

In [ ]:
i=29
print(pred(train_dataset["train"][i]["text"]))
print(train_dataset["train"][i]["output"])
print(train_dataset["train"][i]["text"])

In [ ]:
# import torch
# torch.cuda.empty_cache()
# trainer.train()